In [7]:
import polars as pl
import polars.selectors as cs

In [38]:
disasters = pl.read_csv("1970-2021_DISASTERS.csv", ignore_errors=True).rename(
        {"Disaster Type" : "disasterType"})




ColumnNotFoundError: "len" not found

In [42]:
frequencies = disasters.select(
    ["Year", "disasterType"]
    ).group_by("Year", maintain_order=True).len()

frequencies
# frequencies.write_csv("cleaned/disaster_frequencies.csv")



Year,len
i64,u32
1970,82
1971,63
1972,63
1973,65
1974,72
…,…
2017,371
2018,338
2019,442


In [51]:
annual_country_freq = disasters.select(
    ["Year", "disasterType", "Country"]
    ).group_by(["Year", "Country", "disasterType"], maintain_order=True).len()

# group least frequent disaster types into "other category"
least_freq = annual_country_freq.group_by("disasterType").sum().sort("len")[:6].select("disasterType")
least_freq = least_freq.to_series().to_list()
annual_country_freq = annual_country_freq.with_columns(disasterType = pl.col("disasterType").replace(least_freq, ["Other"] * 6 ))
annual_country_freq.filter(pl.col("disasterType") == "Other")


Year,Country,disasterType,len
i64,str,str,u32
1971,"""Canada""","""Other""",1
1975,"""Costa Rica""","""Other""",1
1975,"""Ecuador""","""Other""",2
1976,"""Costa Rica""","""Other""",1
1976,"""Ecuador""","""Other""",1
…,…,…,…
2021,"""Philippines (the)""","""Other""",1
2021,"""Canary Is""","""Other""",1
2019,"""Uganda""","""Other""",1
